In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import CLIPModel, CLIPProcessor

from torcheeg.models import EEGNet

from tqdm import tqdm

from EEGDataset import EEGDataset

/mnt/D/University/Fall 2025/BCI/Project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "openai/clip-vit-base-patch32"
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# clip_processor = CLIPProcessor.from_pretrained(MODEL_NAME)
# clip_model = CLIPModel.from_pretrained(MODEL_NAME).to(device)

model = SentenceTransformer("all-MiniLM-L6-v2").to(device)

In [4]:
embeddings = model.encode(["hello world", "open source embeddings"])
embeddings.shape, embeddings.dtype, type(embeddings)

((2, 384), dtype('float32'), numpy.ndarray)

In [5]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
ds = EEGDataset("shards", pad_upto=6000)
# ds = EEGDataset("/home/mostafaelfaggal/Documents/BCI", pad_upto=6000)
ds[0][0].shape, ds[0][1]

(torch.Size([3438, 6000]),
 'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.')

In [ ]:
train_ds, val_ds, test_ds = ds.split_train_valid_test(train_ratio=0.7, valid_ratio=0.15, shuffle=False)

train_dl = train_ds.getLoader(batch_size=25, num_workers=0)
val_dl = val_ds.getLoader(batch_size=25, num_workers=0)
test_dl = test_ds.getLoader(batch_size=25, num_workers=0)

In [8]:
embeddings = model.encode([ds[0][1]])
embeddings.shape, embeddings.dtype, type(embeddings)

((1, 384), dtype('float32'), numpy.ndarray)

In [ ]:
ds[0][0].shape

for batch_data, batch_labels in train_dl:
    print(batch_data.shape)
    print(batch_labels)
    break

In [ ]:
class TextEncoder(nn.Module):
    def __init__(self):
        super(TextEncoder, self).__init__()
        self.text_encoder_model = SentenceTransformer("all-MiniLM-L6-v2")

    def forward(self, texts):
        embeddings = self.text_encoder_model.encode(texts, convert_to_tensor=True)
        return embeddings

class LocalizedEEGEncoder(nn.Module):
    def __init__(self, ch_count=8196, embedding_dim=384):
        super(LocalizedEEGEncoder, self).__init__()

        self.temporal = nn.Sequential(
            nn.Conv1d(ch_count, 1024, 11, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(1024, 512, 11, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(512, 256, 11, padding=1),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((256, 1))
        )

        self.fc = nn.Linear(256, embedding_dim)

        # self.eeg_encoder = EEGNet(chunk_size=10000,
        #                     num_electrodes=ch_count,
        #                     dropout=0.3,
        #                     kernel_1=64,
        #                     kernel_2=16,
        #                     F1=8,
        #                     F2=16,
        #                     D=2,
        #                     num_classes=embedding_dim)

    def forward(self, x):
        # x = torch.fft.rfft(x, dim=2)
        # x = torch.log(torch.abs(x) + 1e-8)
        x = self.temporal(x).squeeze(-1)
        x = self.fc(x)

        # x = self.eeg_encoder(x)

        x = F.normalize(x, p=2, dim=1)
        return x
    
class EEGCLIPModel(nn.Module):
    def __init__(self, ch_count=8196, embedding_dim=384, freeze_text=True):
        super(EEGCLIPModel, self).__init__()
        self.text_encoder = TextEncoder()
        self.eeg_encoder = LocalizedEEGEncoder(ch_count=ch_count, embedding_dim=embedding_dim)

        if freeze_text:
            for param in self.text_encoder.parameters():
                param.requires_grad = False

    def forward(self, eeg_data, texts):
        eeg_embeddings = self.eeg_encoder(eeg_data)
        text_embeddings = self.text_encoder(texts)
        return eeg_embeddings, text_embeddings

# Training

In [11]:
model = EEGCLIPModel(3438).to(device)
# model.load_state_dict(torch.load("best_model.pt"))

In [ ]:
def train(model: nn.Module, train_dataloader: DataLoader, valid_loader: DataLoader, epochs: int = 10):
# def train(model: nn.Module, dataset: EEGDataset, epochs: int = 10):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    best_valid_loss = None

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for batch in tqdm(train_dataloader):
        # for i in tqdm(range(40)):
            # batch = dataset[i]
            eeg_data, texts = batch
            # eeg_data = eeg_data.unsqueeze(0)
            # texts = [texts]
            eeg_data = eeg_data.to(torch.float32).to(device)
            texts = list(texts)

            optimizer.zero_grad()
            eeg_embeddings, text_embeddings = model(eeg_data, texts)

            # loss = ((eeg_embeddings - text_embeddings) ** 2).mean()
            loss = 1.0 - F.cosine_similarity(eeg_embeddings, text_embeddings).mean()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        # avg_loss = total_loss / 40


        model.eval()
        total_valid_loss = 0.0
        with torch.inference_mode():
            for batch in tqdm(valid_loader):
            # for i in tqdm(range(40,45)):
                # batch = dataset[i]
                eeg_data, texts = batch
                # eeg_data = eeg_data.unsqueeze(0)
                # texts = [texts]
                eeg_data = eeg_data.to(torch.float32).to(device)
                texts = list(texts)

                eeg_embeddings, text_embeddings = model(eeg_data, texts)

                # loss = ((eeg_embeddings - text_embeddings) ** 2).mean()
                loss = 1.0 - F.cosine_similarity(eeg_embeddings, text_embeddings).mean()

                total_valid_loss += loss.item()
        avg_valid_loss = total_valid_loss / len(valid_loader)
        # avg_valid_loss = total_valid_loss / 5

        if (best_valid_loss is None) or (avg_valid_loss < best_valid_loss):
            print(f"Valid Loss: {avg_valid_loss:.10f}")
            best_valid_loss = avg_valid_loss
            torch.save(model.state_dict(), "best_model.pt")

        print(f"Epoch [{epoch+1}/{epochs}]:- Train Loss: {avg_loss:.6f} | Valid Loss: {avg_valid_loss:.6f}")
        torch.save(model.state_dict(), "last_model.pt")    

        torch.cuda.empty_cache()


In [ ]:
def test(model: nn.Module, test_dataloader: DataLoader):
# def test(model: nn.Module, dataset: EEGDataset):
    model.eval()
    total_loss = 0.0
    with torch.inference_mode():
        for batch in tqdm(test_dataloader):
        # for i in tqdm(range(45, 49)):
            # batch = dataset[i]
            eeg_data, texts = batch
            # eeg_data = eeg_data.unsqueeze(0)
            # texts = [texts]
            eeg_data = eeg_data.to(torch.float32).to(device)
            texts = list(texts)

            eeg_embeddings, text_embeddings = model(eeg_data, texts)

            # loss = ((eeg_embeddings - text_embeddings) ** 2).mean()
            loss = 1.0 - F.cosine_similarity(eeg_embeddings, text_embeddings).mean()

            total_loss += loss.item()
    avg_loss = total_loss / 4
    print(f"Test Loss: {avg_loss:.6f}")

In [ ]:
train(model, train_dl, val_dl, epochs=20)
# train(model, ds, epochs=20)

100%|██████████| 5/5 [00:03<00:00,  1.39it/s]


Valid Loss: 0.7127529979
Epoch [1/20]:- Train Loss: 0.766092 | Valid Loss: 0.712753


100%|██████████| 5/5 [00:03<00:00,  1.48it/s]


Valid Loss: 0.6473903775
Epoch [2/20]:- Train Loss: 0.639115 | Valid Loss: 0.647390


100%|██████████| 5/5 [00:03<00:00,  1.45it/s]


...

Valid Loss: 0.5881092787
Epoch [18/20]:- Train Loss: 0.555975 | Valid Loss: 0.588109


100%|██████████| 5/5 [00:04<00:00,  1.24it/s]


Valid Loss: 0.5880192041
Epoch [19/20]:- Train Loss: 0.555820 | Valid Loss: 0.588019


100%|██████████| 5/5 [00:04<00:00,  1.03it/s]


Valid Loss: 0.5879366636
Epoch [20/20]:- Train Loss: 0.555675 | Valid Loss: 0.587937


In [ ]:
test(model, test_dl)
# test(model, ds)

100%|██████████| 4/4 [00:03<00:00,  1.11it/s]

Test Loss: 0.585270
